**Please only run the following cell if you're running this notebook on Google colab or another cloud hosted notebook server.**

In [0]:
!pip install qiskit

# BB84 Quantum Key Distribution (QKD) Protocol (with eavesdropping)

This notebook is a _demonstration_ of the BB84 Protocol for QKD using Qiskit. 
BB84 is a quantum key distribution scheme developed by Charles Bennett and Gilles Brassard in 1984 ([paper]).
The first three sections of the paper are readable and should give you all the necessary information required. 

[paper]: http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf 


In [0]:
from qiskit import BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit import IBMQ

from qiskit.tools.monitor import job_monitor
%pylab inline

We will now simulate the effects of eavesdropping on BB84. The qubit capabilites of the available backends are listed below:


| Backend | Number of qubits |
|----------|------------------|
|qasm_simulator|24|
|ibmq_qasm_simulator|32|
|ibmqx4|5|
|ibmqx2|5|
|ibmq_16_melbourne|14|

## Choosing bases and encoding states

Alice generates two binary strings. One encodes the basis for each qubit:

$0 \rightarrow$ Computational basis

$1  \rightarrow$ Hadamard basis

The other encodes the state:

$0  \rightarrow|0\rangle$ or $|+\rangle $ 

$1  \rightarrow|1\rangle$  or  $|-\rangle $ 

Bob also generates a binary string and uses the same convention to choose a basis for measurement


In [0]:
num_qubits = 24

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)
eve_basis = np.random.randint(2, size=num_qubits)

print(alice_state)
print(alice_basis)
print(bob_basis)
print(eve_basis)

## Creating the circuit

Based on the following results:

$X|0\rangle = |0\rangle$

$H|0\rangle = |+\rangle$

$ HX|0\rangle = |-\rangle$


Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $X$ gate to the qubit. To encode 0, no action is needed.
2. Wherever she wants to encode it in the Hadamard basis, she applies an $H$ gate. No action is necessary to encode a qubit in the computational basis.

3. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

4. However, Eve intercepts the qubits and measures them by choosing a basis as per her generated random binary string. To measure a qubit in the Hadamard basis, she applies an $H$ gate to the corresponding qubit and then performs a mesurement on the computational basis. 

4. Bob measures the qubits according to his binary string. Bob also measures using the same method as Eve.


The result of Eve's measurements are stored in the first half of the classical registers and the results of Bob's measurements are stored in the second.


In [0]:
q = QuantumRegister(num_qubits,'q')
c_eve = ClassicalRegister(num_qubits,'c_eve')
c_bob = ClassicalRegister(num_qubits,'c_bob')

bb84 = QuantumCircuit(q,c_eve,c_bob,name='bb84')

# Alice prepares her qubits

for index, _ in enumerate(alice_basis):
    if alice_state[index] == 1:
        bb84.x(q[index])
    if alice_basis[index] == 1:
        bb84.h(q[index])
bb84.barrier()  

# Eve chooses her bases
for index, _ in enumerate(eve_basis):
    if eve_basis[index] == 1:
        bb84.h(q[index])
bb84.barrier()  

# Eve measures the qubits
for index, _ in enumerate(eve_basis):
    bb84.measure(q[index],c_eve[index])
bb84.barrier()        

# Bob chooses his bases
for index, _ in enumerate(bob_basis):
    if bob_basis[index] == 1:
        bb84.h(q[index])
bb84.barrier()        
   
# Bob measures the received qubits
for index, _ in enumerate(bob_basis):        
    bb84.measure(q[index],c_bob[index])
        
bb84.barrier()        
    
            
# bb84.draw()


### Running it on the IBMQ machines

The following cell is only needed if you want to use the IBM Q Experience devices. Please enter your API token between the quotes. Instructions are given [here](https://github.com/Qiskit/qiskit/blob/master/docs/install.rst#access-ibm-q-devices)
Uncomment the code in the cell to use it. 

In [0]:
# token = ''
# IBMQ.enable_account(token)
# IBMQ.backends()

## Performing the actual experiment

Run the code on the desired backend. Use the cell above to authenticate yourself to the IBMQ network.

In [0]:
backend = BasicAer.get_backend('qasm_simulator')
# backend = IBMQ.get_backend('ibmq_16_melbourne')
# backend = IBMQ.get_backend('ibmq_qasm_simulator')

num_shots = 1024 # Change this to alter the number of times your circuit runs

job = execute(bb84,backend,shots=num_shots)

job_monitor(job)

result = job.result()
counts = result.get_counts()


bob_keys_temp = []
eve_keys_temp = []


print("Bob's string\t\tEve's string")
for count in [*counts]:
    bob_key = count[2*num_qubits:num_qubits-1:-1]
    eve_key = count[num_qubits:0:-1]
    bob_keys_temp.append(bob_key)
    eve_keys_temp.append(eve_key)
    print(bob_key + '\t' + eve_key)


In [0]:
eve_keys = []
bob_keys = []
for eve_temp_key, bob_temp_key in zip(eve_keys_temp,bob_keys_temp):
    eve_key = ''
    bob_key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]:
            eve_key += str(eve_temp_key[i])
            bob_key += str(bob_temp_key[i])
    eve_keys.append(eve_key)
    bob_keys.append(bob_key)
    
    
print("Bob's key\tEve's key")
for eve_key, bob_key in zip(eve_keys,bob_keys):
    print(bob_key + '\t\t' + eve_key)